In [2]:
import dash
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input,Output
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
from BasketballCourt import get_layout
import json
import plotly.colors as pc
import dash_loading_spinners as dls
from cachetools import cached, TTLCache
import datetime
import getpbp
import jsonlines

In [3]:
import getdata as gd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [4]:
basketball_court_layout = get_layout()

In [5]:
with open('new_data/game_dict.json', "r") as json_file:
    game_dict = json.load(json_file)

In [6]:
all_player_shots = pd.read_csv('new_data/all_player_shots.csv', dtype = {'game_id': str})

In [30]:
all_player_shots.head()

,shotType,ACTIONTYPE,score_miss,eventType,HOMEDESCRIPTION,VISITORDESCRIPTION,gameClock,shotClock,period,x,y,closest_opponent_dist,game_id,height,name,position,Team,closest_opponent,opponent_strength
0,2PT_PAINT,2PT,score,SHOT,Butler 1' Putback Layup (2 PTS),NaN,583.72,9.6,1.0,-41.53,-0.84,2.90,0042100302,79.0,Jimmy Butler,F,home,Marcus Smart,0.0429
1,2PT_DRIBBLE SHOT,2PT,score,DRIBBLE SHOT,Butler 14' Jump Shot (4 PTS),NaN,516.48,10.9,1.0,-32.20,-16.87,3.05,0042100302,79.0,Jimmy Butler,F,home,Marcus Smart,0.0429
2,2PT_DRIBBLE SHOT,2PT,miss,DRIBBLE SHOT,MISS Butler 13' Fadeaway Jumper,NaN,404.96,1.6,1.0,-34.63,12.53,4.21,0042100302,79.0,Jimmy Butler,F,home,Jayson Tatum,0.0486
3,3PT_DRIBBLE SHOT,3PT,miss,DRIBBLE SHOT,MISS Butler 24' 3PT Jump Shot,NaN,366.68,10.1,1.0,-34.78,-22.71,4.42,0042100302,79.0,Jimmy Butler,F,home,Jaylen Brown,0.0489
4,2PT_DRIBBLE SHOT,2PT,miss,DRIBBLE SHOT,MISS Butler 17' Turnaround Fadeaway Shot,NaN,88.48,11.5,1.0,-31.45,-8.85,2.66,0042100302,79.0,Jimmy Butler,F,home,Payton Pritchard,0.0245


In [7]:
all_player_shots['x'] = all_player_shots.apply(lambda row: 1-row['x'] if (row['period'] == 3) | (row['period'] == 4) 
                                                    else row['x'], axis = 1)

In [ ]:
# fig = go.Figure(layout=basketball_court_layout)
# fig.update_layout(width=1410, height=750)

# fig.show()

In [27]:
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = html.Div([
    
    html.Div([
        html.H3('NBA Player Shot Quality and Movement Analysis'),
        dcc.Dropdown(
            id='game-dropdown',
            options=[{'label': k, 'value': k} for k in game_dict.keys()],
            value='06-16-2022 Boston Celtics vs Golden State Warriors'
        )
    ], style={'width': '40%', 'display': 'inline-block','margin-top': '40px', 'margin-left': '180px'}),

    html.Hr(),       
        
    html.Div([
        
        html.Div(id = 'home-team', style={'margin-left': '180px'}),       
        dcc.Dropdown(id='home-player-dropdown', style={'width': '50%', 'margin-left': '90px'}),  
        
        html.Div([

            html.Div(
                dcc.Loading(
                    id="home-heatmap-loading",
                    type="circle",
                    color="steelblue",
                    fullscreen=False,
                    children=[
                        dcc.Graph(id='home-heatmap')
                    ]
                 ), style={'width': '90%', 'vertical-align': 'top', 'margin-left': '20px'}
            ),

            html.Div(
                html.P('OPPONENT BASKET', style={'transform': 'rotate(-90deg)', 'white-space': 'nowrap'}),
                style={'position': 'absolute', 'top': '620px', 'left': '50px'}
            ),

            html.Div(
                html.P('OWN BASKET', style={'transform': 'rotate(-90deg)', 'white-space': 'nowrap'}),
                style={'position': 'absolute', 'top': '620px', 'left': '1250px', 'display': 'inline-block'}
            )
        ])
    ], style={'width': '48%', 'display': 'inline-block', 'margin-left': '10px'}),
    
    html.Hr(style={'width': '1px', 'height': '1020px', 'position': 'absolute','display': 'inline-block',
                   'left': '1440px','background-color': 'black','vertical-align': 'top'}),
    
    html.Div([
        
        html.Div(id = 'away-team', style={'margin-left': '180px'}),
        dcc.Dropdown(id='away-player-dropdown', style={'width': '50%', 'margin-left': '90px'}),

        html.Div([

            html.Div(
                dcc.Loading(
                    id="away-heatmap-loading",
                    type="circle",
                    color="steelblue",
                    fullscreen=False,
                    children=[
                        dcc.Graph(id='away-heatmap')
                    ]
                 ), style={'width': '90%', 'vertical-align': 'top', 'margin-left': '20px'}
            ),

            html.Div(
                html.P('OWN BASKET', style={'transform': 'rotate(-90deg)', 'white-space': 'nowrap'}),
                style={'position': 'absolute', 'top': '620px', 'left': '1500px'}
            ),

            html.Div(
                html.P('OPPONENT BASKET', style={'transform': 'rotate(-90deg)', 'white-space': 'nowrap'}),
                style={'position': 'absolute', 'top': '620px', 'left': '2650px', 'display': 'inline-block'}
            )
        ])      
    ], style={'width': '48%', 'display': 'inline-block', 'margin-left': '60px'})

 ])

@app.callback(
    Output('home-team', 'children'),
    [Input('game-dropdown', 'value')]
)
def set_team_value(selected_game):
    home_team = game_dict[selected_game]['home_team']
    return html.Div([
        html.H4(f"Home Team: {home_team}"),
    ])   

@app.callback(
    Output('home-player-dropdown', 'options'),
    [Input('game-dropdown', 'value')])
def set_player_options(selected_game):
    return [{'label': i, 'value': i} for i in game_dict[selected_game]['home_player']]

@app.callback(
    Output('home-player-dropdown', 'value'),
    [Input('home-player-dropdown', 'options')])
def set_player_value(available_options):
    return available_options[0]['value']

@app.callback(
    Output('home-heatmap', 'figure'),
    [Input('home-player-dropdown', 'value'),
     Input('game-dropdown', 'value')]
)

def update_figure_home(player, game):
    date = game.split()[0]
    nbaId = gd.get_nbaId(date)
    playerId = gd.get_playerInfo(player, 'fullName', 'id')
    df = gd.get_xy_df(nbaId, playerId)
    mapping = {1:'First Quarter', 2:'Second Quarter', 3:'Third Quarter', 4:'Forth Quarter'}
    df['period'] = df['period'].map(mapping)  
    
    periods = df['period'].unique()
    colors = {'First Quarter':'blues', 'Second Quarter':'blues', 'Third Quarter':'ylorbr', 'Forth Quarter':'ylorbr'}
    
    df = df.iloc[::4].reset_index(drop=True)
    xy_coords = np.vstack([df.x, df.y])
    kernel = stats.gaussian_kde(xy_coords)
    df['density'] = kernel(xy_coords)
    
    df_p = df[df['period'] == periods[0]]
    df_shot = all_player_shots[(all_player_shots['name'] == player) & (all_player_shots['game_id'] == nbaId)] 
    
    fig = go.Figure(layout=basketball_court_layout)
#     fig.add_trace(go.Scatter(
#             x=df_p['x'],
#             y=df_p['y'],
#             name = str(periods[0]),
#             mode='markers',
#             marker=dict(
#                 color=df_p['density'],
#                 colorscale=colors[periods[0]][:8],
#                 size=10,
#                 opacity=0.25,
#                 symbol='hexagon'
#             ), legend = 'legend1',
#             showlegend = True
#         ))  
    
#     if len(periods) == 1:
#         pass
#     else:
#         for period in periods[1:]:
#             df_p = df[df['period'] == period]
#             fig.add_trace(go.Scatter(
#                 x=df_p['x'],
#                 y=df_p['y'],
#                 name = str(period),
#                 mode='markers',
#                 marker=dict(
#                     color=df_p['density'],
#                     colorscale=colors[period][:8],
#                     size=10,
#                     opacity=0.25,
#                     symbol='hexagon'
#                 ), legend = 'legend1',
#                 showlegend = True, visible='legendonly'
#             ))

#     fig.update_layout(legend1=dict(orientation='h', yanchor='top', y=1.06, xanchor = 'left', x=0.1, font = dict(size = 16)))
    
    
#     shot_trace = px.scatter(df_shot, x='x', y='y', color='score_miss',
#                  color_discrete_map={'score': 'darkred', 'miss': 'darkred'},
#                  symbol = 'score_miss',
#                  symbol_map={'score': 'circle', 'miss': 'circle-open'},
#                  )
#     shot_trace.update_traces(marker=dict(size=14))
#     fig.add_traces(shot_trace.data)
    
    fig.update_layout(width=1410, height=750)
    return fig

@app.callback(
    Output('away-team', 'children'),
    [Input('game-dropdown', 'value')]
)
def set_team_value(selected_game):
    away_team = game_dict[selected_game]['away_team']
    return html.Div([
        html.H4(f"Away Team: {away_team}"),
    ])  

@app.callback(
    Output('away-player-dropdown', 'options'),
    [Input('game-dropdown', 'value')])
def set_player_options(selected_game):
    return [{'label': i, 'value': i} for i in game_dict[selected_game]['away_player']]

@app.callback(
    Output('away-player-dropdown', 'value'),
    [Input('away-player-dropdown', 'options')])
def set_player_value(available_options):
    return available_options[0]['value']

@app.callback(
    Output('away-heatmap', 'figure'),
    [Input('away-player-dropdown', 'value'),
     Input('game-dropdown', 'value')]
)

def update_figure_away(player, game):
    date = game.split()[0]
    nbaId = gd.get_nbaId(date)
    playerId = gd.get_playerInfo(player, 'fullName', 'id')
    df = gd.get_xy_df(nbaId, playerId, home = False)
    mapping = {1:'First Quarter', 2:'Second Quarter', 3:'Third Quarter', 4:'Forth Quarter'}
    df['period'] = df['period'].map(mapping)  
    
    periods = df['period'].unique()
    colors = {'First Quarter':'blues', 'Second Quarter':'blues', 'Third Quarter':'ylorbr', 'Forth Quarter':'ylorbr'}
    
    df = df.iloc[::4].reset_index(drop=True)
    xy_coords = np.vstack([df.x, df.y])
    kernel = stats.gaussian_kde(xy_coords)
    df['density'] = kernel(xy_coords)
      
    df_p = df[df['period'] == periods[0]]
    df_shot = all_player_shots[(all_player_shots['name'] == player) & (all_player_shots['game_id'] == nbaId)] 
    
    fig = go.Figure(layout=basketball_court_layout)
#     fig.add_trace(go.Scatter(
#             x=df_p['x'],
#             y=df_p['y'],
#             name = str(periods[0]),
#             mode='markers',
#             marker=dict(
#                 color=df_p['density'],
#                 colorscale=colors[periods[0]][:8],
#                 size=10,
#                 opacity=0.25,
#                 symbol='hexagon'
#             ), legend = 'legend1',
#             showlegend = True
#         ))  
    
#     if len(periods) == 1:
#         pass
#     else:
#         for period in periods[1:]:
#             df_p = df[df['period'] == period]
#             fig.add_trace(go.Scatter(
#                 x=df_p['x'],
#                 y=df_p['y'],
#                 name = str(period),
#                 mode='markers',
#                 marker=dict(
#                     color=df_p['density'],
#                     colorscale=colors[period][:8],
#                     size=10,
#                     opacity=0.25,
#                     symbol='hexagon'
#                 ), legend = 'legend1',
#                 showlegend = True, visible='legendonly'
#             ))

#     fig.update_layout(legend1=dict(orientation='h', yanchor='top', y=1.06, xanchor = 'left', x=0.1, font = dict(size = 16)))

    
#     shot_trace = px.scatter(df_shot, x='x', y='y', color='score_miss',
#                  color_discrete_map={'score': 'darkred', 'miss': 'darkred'},
#                  symbol = 'score_miss',
#                  symbol_map={'score': 'circle', 'miss': 'circle-open'},
#                  )
#     shot_trace.update_traces(marker=dict(size=14))
    
#    fig.add_traces(shot_trace.data)  
    fig.update_layout(width=1410, height=750)        
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, mode='external', port=1020)

In [ ]:
gd.get_playerInfo('Marcus Smart', 'fullName', 'id')

In [ ]:
df = gd.get_xy_df('0042100406', 'ff411907-89ef-11e6-9c68-a45e60e298d3')

In [ ]:
df.head()